In [191]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, SimpleRNN

In [5]:
file_path = "Seq8_Nov2023.xlsx"
df = pd.read_excel(file_path)

In [6]:
df

,Sequence,Dist 1,Bind 1,Dist 2,Bind 2,Dist 3,Bind 3,Ave. dist,std. Dist,Ave. bind,Std. Bind
0,WFDGRAPR,8.30,-6.9,7.43,-6.4,5.41,-6.7,7.046667,1.482644,-6.666667,0.251661
1,PPSPRPRR,5.69,-6.6,8.78,-6.2,4.83,-7.1,6.433333,2.077266,-6.633333,0.450925
2,RGGGRPPR,4.85,-6.5,6.78,-6.9,5.67,-6.4,5.766667,0.968624,-6.600000,0.264575
3,ASFSRAKR,5.37,-7.0,4.82,-6.7,4.80,-5.8,4.996667,0.323471,-6.500000,0.624500
4,TRPRRPPR,5.56,-6.3,4.82,-6.5,5.56,-6.7,5.313333,0.427239,-6.500000,0.200000
...,...,...,...,...,...,...,...,...,...,...,...
2457,DKSRKIAR,4.82,-4.2,4.81,-4.5,7.79,-4.8,5.806667,1.717624,-4.500000,0.300000
2458,VCKKRQHR,5.46,-4.9,5.17,-4.0,5.42,-4.5,5.350000,0.157162,-4.466667,0.450925
2459,SSKRRKRR,6.68,-4.8,4.82,-3.9,4.83,-4.7,5.443333,1.070996,-4.466667,0.493288
2460,KKSKKSSR,4.88,-4.6,5.82,-4.4,4.77,-4.3,5.156667,0.577090,-4.433333,0.152753


In [26]:
encoded_data = pd.get_dummies(df['Sequence'].apply(list).explode(), prefix='', prefix_sep='')
df['Sequence'].apply(list).explode().unique()

array(['W', 'F', 'D', 'G', 'R', 'A', 'P', 'S', 'K', 'T', 'Q', 'V', 'Y',
       'L', 'I', 'H', 'E', 'N', 'C', 'M'], dtype=object)

In [211]:
amino_acids = sorted(['W', 'F', 'D', 'G', 'R', 'A', 'P', 'S', 'K', 'T', 'Q', 'V', 'Y', 'L', 'I', 'H', 'E', 'N', 'C', 'M'])

In [212]:
def encode_sequence(seq):
    temp = [[0]*20 for i in range(8)]
    for i in range(8):
        for j in range(20):
            if seq[i] == amino_acids[j]:
                temp[i][j] = 1
    return temp

In [213]:
df['encoded'] = df.apply(lambda row: encode_sequence(row['Sequence']), axis=1)
df['encoded'][0]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]

In [217]:
x = df['encoded']
y = df['Ave. bind']

In [220]:
# Reshape the encoded sequences
encoded_sequence_array = np.array(x)
num_samples = len(encoded_sequence_array)
x_reshaped = np.empty((num_samples, 8, 20))
for i in range(num_samples):
    x_reshaped[i] = encoded_sequence_array[i]
    
print("Shape of reshaped data:", x_reshaped.shape)

Shape of reshaped data: (2462, 8, 20)


In [242]:
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y, test_size=0.20, random_state=502)

In [263]:
# LSTM implementation
model = Sequential()
model.add(LSTM(50, input_shape=(8, x_reshaped.shape[2])))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mae')
model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/50
31/31 [==============================] - 1s 12ms/step - loss: 4.9449 - val_loss: 3.6362
Epoch 2/50
31/31 [==============================] - 0s 4ms/step - loss: 1.1255 - val_loss: 0.3485
Epoch 3/50
31/31 [==============================] - 0s 4ms/step - loss: 0.3136 - val_loss: 0.2893
Epoch 4/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2859 - val_loss: 0.2799
Epoch 5/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2764 - val_loss: 0.2740
Epoch 6/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2658 - val_loss: 0.2656
Epoch 7/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2500 - val_loss: 0.2542
Epoch 8/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2400 - val_loss: 0.2486
Epoch 9/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2358 - val_loss: 0.2399
Epoch 10/50
31/31 [==============================] - 0s 4ms/step - loss: 0.2273 - val_loss: 0.2380
Epoch 11/50
31/31 

In [257]:
# RNN implementation
model = Sequential()

model.add(SimpleRNN(units=50, input_shape=(8, x_reshaped.shape[2])))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mae')

model.fit(x_train, y_train, epochs=50, batch_size=64, validation_data=(x_test, y_test))


Epoch 1/50
31/31 [==============================] - 1s 6ms/step - loss: 3.4834 - val_loss: 1.4571
Epoch 2/50
31/31 [==============================] - 0s 2ms/step - loss: 0.5854 - val_loss: 0.3323
Epoch 3/50
31/31 [==============================] - 0s 2ms/step - loss: 0.3033 - val_loss: 0.2911
Epoch 4/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2959 - val_loss: 0.2902
Epoch 5/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2945 - val_loss: 0.2900
Epoch 6/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2939 - val_loss: 0.2896
Epoch 7/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2936 - val_loss: 0.2883
Epoch 8/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2928 - val_loss: 0.2873
Epoch 9/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2911 - val_loss: 0.2857
Epoch 10/50
31/31 [==============================] - 0s 2ms/step - loss: 0.2896 - val_loss: 0.2842
Epoch 11/50
31/31 [

In [264]:
# Evaluate the model
mean_error = model.evaluate(x_test, y_test)
print(f"Mean Absolute Error on Test Set: {mean_error}")

16/16 [==============================] - 0s 994us/step - loss: 0.2307
Mean Absolute Error on Test Set: 0.23068930208683014


In [265]:
# Predict on the test set
predictions = model.predict(x_test)

16/16 [==============================] - 0s 1ms/step


In [266]:
test_df = pd.DataFrame()
test_df['encoded'] = x_test.tolist()
test_df['predictions'] = predictions

In [254]:
def decode_sequence(encoded_seq):
    decoded_seq = ""
    for i in range(len(encoded_seq)):
        # Find the index of 1 in each row of the encoded sequence
        index = encoded_seq[i].index(1)
        # Use the index to get the corresponding amino acid from the encoding list
        decoded_seq += amino_acids[index]
    return decoded_seq


In [267]:
test_df['Sequence'] = test_df['encoded'].apply(decode_sequence)

In [268]:
test_df

,encoded,predictions,Sequence
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.799766,PQALRTTR
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.492521,YISKRSKR
2,"[[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.750213,DLLGHARR
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.810551,TPLPRTRR
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.836238,PVDARHAR
...,...,...,...
488,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.580463,TKLTRPVR
489,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,...",-5.597978,KEPARHRR
490,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.275733,NQPKRKKR
491,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",-5.348962,LERMIRRR
